In [2]:
# Paquetería
# Herramientas matemáticas
import numpy as np

# Lectura de .csv a Dataframes
import pandas as pd

# Graficas
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

# Herramienta para hacer copias de Dataframes sin conflictos
import copy

import sys
###############################################################
#####                       ¡WARNING!                       ###
############       En caso de que exista un error        ######
############    Probar quitar el punto en la ruta     #########
############################################################### 

dot = '.'
# Correccion:
#dot = ''
sys.path.append(dot+'./utils/')
output_path = dot+'./output/'
from utils import *

# 0. Lectura de Ventas_v1.csv

In [3]:
Ventas_v2 = pd.read_csv(output_path+'Ventas_v1.csv')
Ventas_v2.shape

(2564361, 5)

Verificamos los registros unicos

In [4]:
Ventas_v2["Flight_ID"].nunique()

108964

# 1. Pivot Table

Creación de nuevas columnas a partir de ProductType	 y ProductName

In [6]:
Ventas_v2['ProductType'].unique()

array(['Botanas', 'Licores', 'Vivabus', 'Transportaciones CUN',
       'Galletas', 'Specials', 'Bebidas Calientes', 'COMBOS CREW',
       'Hertz.', 'OFERTAS ', 'Transportaciones MTY',
       'Transportaciones TLC', 'VIVA PLAY', 'VIVA Taxis', 'Antros',
       'VivaTransfer', 'Alimentos Charter', 'Perecederos', 'Refrescos',
       'Sopas', 'Lacteos'], dtype=object)

Eliminamos los tipos de productos que no nos interesan

In [7]:
to_exclude = ['Vivabus', 'Transportaciones CUN', 'Specials', 'COMBOS CREW', 
              'Hertz.', 'OFERTAS ', 'Transportaciones MTY', 'Transportaciones TLC', 
              'VIVA PLAY', 'VIVA Taxis', 'Antros', 'VivaTransfer', 'Alimentos Charter']

sales_data_v2 = Ventas_v2[~Ventas_v2['ProductType'].isin(to_exclude)]

# Reduciccion de la base
sales_data_v2.shape

(2108292, 5)

Ahora, vamos a quedarnos con un registro unico para cada vuelo y a transformar los distintos valores en ProductType en columnas

In [8]:
product_type = sales_data_v2.pivot_table(index='Flight_ID', columns='ProductType', values='Quantity', 
                                          aggfunc='sum', fill_value=0).reset_index()
product_type.columns

Index(['Flight_ID', 'Bebidas Calientes', 'Botanas', 'Galletas', 'Lacteos',
       'Licores', 'Perecederos', 'Refrescos', 'Sopas'],
      dtype='object', name='ProductType')

Mismo caso con ProductName

In [9]:
product_name = sales_data_v2.pivot_table(index='Flight_ID', columns='ProductName', values='Quantity', 
                                          aggfunc='sum', fill_value=0).reset_index()
product_name

ProductName                         Flight_ID  Agua Natural 600 Ml  \
0            00004a718edba9d9ef878d08f02ae057                    4   
1            0000cd79c0c3a9c309df6064dcacaeea                    2   
2            000163f0df9cbfc35c4c06645ec512f6                    0   
3            00017be73003a570dd426b155762769c                    8   
4            0001a43836c338f8d8650aefb11672c9                    3   
...                                       ...                  ...   
108792       fffe47f84ecc55da94b2907a7317dd12                   15   
108793       fffe4a9c19ae2320e76f02939fdff957                    4   
108794       ffff138536f249f69340e0a8336f94a4                    5   
108795       ffff782f2ae79e385a016c00fbd994b8                    3   
108796       ffff881a59bc5274e97ea3c4de344e05                    0   

ProductName  Amstel Ultra  Arandano  Arandano Mango Mix  Arcoiris  Baileys  \
0                       0         0                   0         0        0   
1                       0         0                   0         0        0   
2                       0         0                   0         0        0   
3                       0         0                   1         0        0   
4                       0         0                   2         0        2   
...                   ...       ...                 ...       ...      ...   
108792                  0         0                   0         1        0   
108793                  4         0                   0         0        0   
108794                  0         0                   3         0        0   
108795                  0         0                   0         0        0   
108796                  0         0                   0         0        0   

ProductName  Baileys   Cafe 19 Cafe Clasico  Cafe 19 Capuchino  ...  \
0                   0                     0                  0  ...   
1                   0                     0                  1  ...   
2                   0                     0                  1  ...   
3                   0                     0                  2  ...   
4                   0                     0                  0  ...   
...               ...                   ...                ...  ...   
108792              0                     0                  2  ...   
108793              1                     0                  0  ...   
108794              0                     0                  0  ...   
108795              0                     0                  0  ...   
108796              0                     0                  2  ...   

ProductName  Topochico Seltzer Fresa-Guayaba  Topochico Seltzer Mango  \
0                                          0                        0   
1                                          0                        0   
2                                          0                        0   
3                                          1                        1   
4                                          0                        0   
...                                      ...                      ...   
108792                                     0                        0   
108793                                     0                        0   
108794                                     0                        1   
108795                                     0                        0   
108796                                     0                        0   

ProductName  Tostitos  Tostitos Nachos Con Dip  Ultra Seltzer Frambuesa  \
0                   0                        0                        0   
1                   0                        0                        0   
2                   0                        0                        0   
3                   1                        0                        0   
4                   2                        0                        0   
...               ...                     

Unimos las bases.

In [10]:
sales_data_v3 = pd.merge(product_name, product_type, on='Flight_ID', how='inner')
sales_data_v3

Flight_ID  Agua Natural 600 Ml  Amstel Ultra  \
0       00004a718edba9d9ef878d08f02ae057                    4             0   
1       0000cd79c0c3a9c309df6064dcacaeea                    2             0   
2       000163f0df9cbfc35c4c06645ec512f6                    0             0   
3       00017be73003a570dd426b155762769c                    8             0   
4       0001a43836c338f8d8650aefb11672c9                    3             0   
...                                  ...                  ...           ...   
108792  fffe47f84ecc55da94b2907a7317dd12                   15             0   
108793  fffe4a9c19ae2320e76f02939fdff957                    4             4   
108794  ffff138536f249f69340e0a8336f94a4                    5             0   
108795  ffff782f2ae79e385a016c00fbd994b8                    3             0   
108796  ffff881a59bc5274e97ea3c4de344e05                    0             0   

        Arandano  Arandano Mango Mix  Arcoiris  Baileys  Baileys   \
0              0                   0         0        0         0   
1              0                   0         0        0         0   
2              0                   0         0        0         0   
3              0                   1         0        0         0   
4              0                   2         0        2         0   
...          ...                 ...       ...      ...       ...   
108792         0                   0         1        0         0   
108793         0                   0         0        0         1   
108794         0                   3         0        0         0   
108795         0                   0         0        0         0   
108796         0                   0         0        0         0   

        Cafe 19 Cafe Clasico  Cafe 19 Capuchino  ...  Xx Lager  Xx Ultra  \
0                          0                  0  ...         1         0   
1                          0                  1  ...         1         1   
2                          0                  1  ...         0         0   
3                          0                  2  ...         6         0   
4                          0                  0  ...         0         0   
...                      ...                ...  ...       ...       ...   
108792                     0                  2  ...         1         0   
108793                     0                  0  ...         1         0   
108794                     0                  0  ...         1         2   
108795                     0                  0  ...         0         0   
108796                     0                  2  ...         0         0   

        Bebidas Calientes  Botanas  Galletas  Lacteos  Licores  Perecederos  \
0                       0       13         2        0        1            4   
1                       3       13         1        1        2            1   
2                       2        1         0        0        0            2   
3                       6       39         5        2       17           11   
4                       2       29         5        0       19           10   
...                   ...      ...       ...      ...      ...          ...   
108792                  3       38         3        0        4            0   
108793                  1       21         0        0       12            3   
108794                  2       29         3        0        9            6   
108795                  0       13         0        0       10            3   
108796                  3        2         1        0        0            3   

        Refrescos  Sopas  
0              16      3  
1              17      4  
2               1      0  
3              37     12  
4              34     13  
...           ...    ...  
108792         48      9  
108793         21      3  
108794         27      4  
108795         13      0  
108796          3      0  

[108797 rows x 107 columns]

# 2. Union con la base de viajes tratada Vuelos_2023_v2

In [11]:
Vuelos_2023_v2 = pd.read_csv(output_path + "Vuelos_2023_v2.csv")
print(Vuelos_2023_v2.columns)

Index(['Flight_ID', 'Aeronave', 'DepartureStation', 'ArrivalStation',
       'Destination_Type', 'Origin_Type', 'STD', 'STA', 'Capacity',
       'Passengers', 'Bookings', 'Horario', 'Periodo', 'TiempoVuelo',
       'Dia_Semana', 'Mapeo_Dia_Semana', 'Mapeo_Destination_Type',
       'Mapeo_Origin_Type', 'AbordajeReal'],
      dtype='object')


In [12]:
sales_flight_data = pd.merge(sales_data_v3, Vuelos_2023_v2, on='Flight_ID', how='right')
sales_flight_data.fillna(0, inplace=True)

sales_flight_data

Flight_ID  Agua Natural 600 Ml  Amstel Ultra  \
0       ab954014077430bd842cfa305a55c0f8                  5.0           2.0   
1       efd86c996035dacdca7a0ccb2560dda1                  3.0           0.0   
2       dd0fad3248951d2f71d63e6279aeaa4b                  0.0           0.0   
3       d0987ee648eea254063bfe2b39571b67                  2.0           0.0   
4       3b5df8805161ea827d2f2e4298c38e06                  0.0           0.0   
...                                  ...                  ...           ...   
118540  a23ace5230c863493686a1d369ebc9c0                 30.0           0.0   
118541  9bbd9f3a2bbcaeae864d17e3ee544d2c                  2.0           0.0   
118542  d9ef769565064b1ff1ad90d6d18f69cb                  7.0           0.0   
118543  598035669a4f0fa2ce1207977932df9e                 12.0           0.0   
118544  9c8970e9836d9c5ef9415bfa93c3f408                  9.0           3.0   

        Arandano  Arandano Mango Mix  Arcoiris  Baileys  Baileys   \
0            0.0                 0.0       0.0      0.0       0.0   
1            1.0                 0.0       0.0      0.0       0.0   
2            0.0                 0.0       0.0      0.0       0.0   
3            0.0                 0.0       1.0      0.0       0.0   
4            0.0                 0.0       0.0      0.0       0.0   
...          ...                 ...       ...      ...       ...   
118540       0.0                 1.0       4.0      0.0       0.0   
118541       0.0                 0.0       0.0      0.0       0.0   
118542       0.0                 1.0       2.0      0.0       0.0   
118543       0.0                 1.0       1.0      0.0       0.0   
118544       0.0                 0.0       0.0      1.0       0.0   

        Cafe 19 Cafe Clasico  Cafe 19 Capuchino  ...  Passengers  Bookings  \
0                        0.0                1.0  ...       229.0     157.0   
1                        0.0                0.0  ...       197.0     109.0   
2                        0.0                0.0  ...       200.0     142.0   
3                        0.0                1.0  ...       162.0      90.0   
4                        0.0                0.0  ...       183.0     125.0   
...                      ...                ...  ...         ...       ...   
118540                   0.0                0.0  ...       224.0     105.0   
118541                   0.0                1.0  ...       191.0     119.0   
118542                   0.0                0.0  ...       166.0      89.0   
118543                   0.0                0.0  ...       142.0      54.0   
118544                   0.0                0.0  ...       163.0      95.0   

        Horario  Periodo  TiempoVuelo  Dia_Semana  Mapeo_Dia_Semana  \
0             1        0        165.0    Thursday                 0   
1             0        0        240.0      Monday                 1   
2             2        0         40.0      Monday                 1   
3             1        0         70.0      Friday                 2   
4             2        0         55.0    Thursday                 0   
...         ...      ...          ...         ...               ...   
118540        1        1        205.0      Friday                 2   
118541        1        0        200.0   Wednesday                 4   
118542        0        1        230.0     Tuesday                 3   
118543        1        1        205.0    Thursday                 0   
118544        1        1        200.0      Friday                 2   

        Mapeo_Destination_Type  Mapeo_Origin_Type  AbordajeReal  
0                            0                  0         229.0  
1                            1                  1         186.0  
2                            1                  0         200.0  
3                            2                  0         162.0  
4                            2                  2         183.0  
...                        ...                ...           ...  
1

# 3. Creación de variable Total_Sales

In [13]:
sales_flight_data['Total_Sales'] = sales_flight_data.iloc[:, 99:107].sum(axis=1)
sales_flight_data

Flight_ID  Agua Natural 600 Ml  Amstel Ultra  \
0       ab954014077430bd842cfa305a55c0f8                  5.0           2.0   
1       efd86c996035dacdca7a0ccb2560dda1                  3.0           0.0   
2       dd0fad3248951d2f71d63e6279aeaa4b                  0.0           0.0   
3       d0987ee648eea254063bfe2b39571b67                  2.0           0.0   
4       3b5df8805161ea827d2f2e4298c38e06                  0.0           0.0   
...                                  ...                  ...           ...   
118540  a23ace5230c863493686a1d369ebc9c0                 30.0           0.0   
118541  9bbd9f3a2bbcaeae864d17e3ee544d2c                  2.0           0.0   
118542  d9ef769565064b1ff1ad90d6d18f69cb                  7.0           0.0   
118543  598035669a4f0fa2ce1207977932df9e                 12.0           0.0   
118544  9c8970e9836d9c5ef9415bfa93c3f408                  9.0           3.0   

        Arandano  Arandano Mango Mix  Arcoiris  Baileys  Baileys   \
0            0.0                 0.0       0.0      0.0       0.0   
1            1.0                 0.0       0.0      0.0       0.0   
2            0.0                 0.0       0.0      0.0       0.0   
3            0.0                 0.0       1.0      0.0       0.0   
4            0.0                 0.0       0.0      0.0       0.0   
...          ...                 ...       ...      ...       ...   
118540       0.0                 1.0       4.0      0.0       0.0   
118541       0.0                 0.0       0.0      0.0       0.0   
118542       0.0                 1.0       2.0      0.0       0.0   
118543       0.0                 1.0       1.0      0.0       0.0   
118544       0.0                 0.0       0.0      1.0       0.0   

        Cafe 19 Cafe Clasico  Cafe 19 Capuchino  ...  Bookings  Horario  \
0                        0.0                1.0  ...     157.0        1   
1                        0.0                0.0  ...     109.0        0   
2                        0.0                0.0  ...     142.0        2   
3                        0.0                1.0  ...      90.0        1   
4                        0.0                0.0  ...     125.0        2   
...                      ...                ...  ...       ...      ...   
118540                   0.0                0.0  ...     105.0        1   
118541                   0.0                1.0  ...     119.0        1   
118542                   0.0                0.0  ...      89.0        0   
118543                   0.0                0.0  ...      54.0        1   
118544                   0.0                0.0  ...      95.0        1   

        Periodo  TiempoVuelo  Dia_Semana  Mapeo_Dia_Semana  \
0             0        165.0    Thursday                 0   
1             0        240.0      Monday                 1   
2             0         40.0      Monday                 1   
3             0         70.0      Friday                 2   
4             0         55.0    Thursday                 0   
...         ...          ...         ...               ...   
118540        1        205.0      Friday                 2   
118541        0        200.0   Wednesday                 4   
118542        1        230.0     Tuesday                 3   
118543        1        205.0    Thursday                 0   
118544        1        200.0      Friday                 2   

        Mapeo_Destination_Type  Mapeo_Origin_Type  AbordajeReal  Total_Sales  
0                            0                  0         229.0         42.0  
1                            1                  1         186.0         15.0  
2                            1                  0         200.0          0.0  
3                            2                  0         162.0         22.0  
4                            2                  2         183.0          2.0  
...                        ...                ...           ...          ...  
118540                       2                  0     

In [ ]:
sales_flight_data.to_csv(output_path+'Ventas_Vuelos_2023.csv.csv',index=False)